In [99]:
import sys
import numpy as np
import pandas as pd
#import sklearn
#import pickle

In [100]:
cgmDataFile = './Project-1-Files/CGMData.csv'
insDataFile = './Project-1-Files/InsulinData.csv'

STEP 1: Read data along with parsed dates

In [101]:
cgmData = pd.read_csv(cgmDataFile, parse_dates=[['Date', 'Time']])#, keep_date_col=True)
insData = pd.read_csv(insDataFile, parse_dates=[['Date', 'Time']])#, keep_date_col=True)

C:\Users\abhis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\abhis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (13,43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [102]:
# focus only on datetime and Sensor Glucose data columns
cgmData = cgmData[['Date_Time', 'Sensor Glucose (mg/dL)']]

STEP 2: create 6 columns specifying each type of hyperglycemia ranges for that row's CGM value

In [103]:
# Different types of Hyperglycemia ranges
HGC_Ranges = ['HGC', 'HGC_C', 'HGC_R1', 'HGC_R2', 'HGC_L1', 'HGC_L2']
# TYPE 1: Hyperglycemia (HGC)
cgmData[HGC_Ranges[0]] = cgmData['Sensor Glucose (mg/dL)'] > 180
# TYPE 2: Hyperglycemia Critical (HGC_C)
cgmData[HGC_Ranges[1]] = cgmData['Sensor Glucose (mg/dL)'] > 250
# TYPE 3: Hyperglycemia Range 1 (HGC_R1)
cgmData[HGC_Ranges[2]] = cgmData['Sensor Glucose (mg/dL)'].between(70, 180, inclusive='both')
# TYPE 4: Hyperglycemia Range 2 (HGC_R2)
cgmData[HGC_Ranges[3]] = cgmData['Sensor Glucose (mg/dL)'].between(70, 150, inclusive='both')
# TYPE 5: Hyperglycemia Level 1 (HGC_L1)
cgmData[HGC_Ranges[4]] = cgmData['Sensor Glucose (mg/dL)'] < 70
# TYPE 6: Hyperglycemia Level 2 (HGC_L2)
cgmData[HGC_Ranges[5]] = cgmData['Sensor Glucose (mg/dL)'] < 54

STEP 3: look for 'AUTO MODE ACTIVE PLGM OFF' in insData and get corresponding Date_Time

In [104]:
# D_T_A for Date, Time, Alarm
insData_D_T_A = insData[['Date_Time', 'Alarm']]

In [105]:
# get the 'AUTO MODE ACTIVE PLGM OFF' date and timestamp
autoOnData = insData_D_T_A[insData_D_T_A['Alarm'] == 'AUTO MODE ACTIVE PLGM OFF']
# get the earliest Date_Time value index
autoOnIndex = autoOnData['Date_Time'].idxmin()
# index the earliest DateTime
autoOnDateTime = insData_D_T_A.iloc[autoOnIndex]['Date_Time']

STEP 4: handle missing/NaN data (USING deletion of the entire day if NaN present)

In [106]:
# filter the data
cgmData['Valid_Day'] = cgmData.groupby(pd.Grouper(key='Date_Time', freq='1D'))['Sensor Glucose (mg/dL)'].transform(lambda x: True if (x.count() >= 231 and x.count() <= 288) else False)
cgmData_filt = cgmData[cgmData['Valid_Day']].copy(deep=True)

STEP 5: get corresponding Manual/Auto data based on the autoOnDateTime

In [107]:
# times BEFORE autoOnDateTime are MANUAL MODE
cgmData_D_T_R_Man = cgmData_filt[ cgmData_filt['Date_Time'] < autoOnDateTime ].copy(deep=True)

In [108]:
# times AFTER/ON autoOnDateTime are AUTO MODE
cgmData_D_T_R_Aut = cgmData_filt[ cgmData_filt['Date_Time'] >= autoOnDateTime ].copy(deep=True)

In [109]:
# quick check to see if data extracted is labelled correctly
if (cgmData_D_T_R_Man['Date_Time'].max() <= autoOnDateTime < cgmData_D_T_R_Aut['Date_Time'].min()) is False:
    print("ERROR: The extracted parts Manual and Auto DO NOT make sense chronologically.")

STEP 6: extract data for different segments and sub-segments of the day

In [110]:
# MANUAL data
cgm_OvrNight_Man = cgmData_D_T_R_Man[(cgmData_D_T_R_Man['Date_Time'].dt.hour >= 0) & 
                  (cgmData_D_T_R_Man['Date_Time'].dt.hour < 6)].copy(deep=True)
cgm_DayTime_Man = cgmData_D_T_R_Man[(cgmData_D_T_R_Man['Date_Time'].dt.hour >= 6) & 
                  (cgmData_D_T_R_Man['Date_Time'].dt.hour <= 23)].copy(deep=True)
cgm_WholeDay_Man = cgmData_D_T_R_Man.copy(deep=True)

# AUTO data
cgm_OvrNight_Aut = cgmData_D_T_R_Aut[(cgmData_D_T_R_Aut['Date_Time'].dt.hour >= 0) & 
                  (cgmData_D_T_R_Aut['Date_Time'].dt.hour < 6)].copy(deep=True)
cgm_DayTime_Aut = cgmData_D_T_R_Aut[(cgmData_D_T_R_Aut['Date_Time'].dt.hour >= 6) & 
                  (cgmData_D_T_R_Aut['Date_Time'].dt.hour <= 23)].copy(deep=True)
cgm_WholeDay_Aut = cgmData_D_T_R_Aut.copy(deep=True)

In [146]:
cgm_DayTime_Aut[(cgm_DayTime_Aut['Date_Time'].dt.day == 9) & (cgm_DayTime_Aut['Date_Time'].dt.month == 8)]

,Date_Time,Sensor Glucose (mg/dL),HGC,HGC_C,HGC_R1,HGC_R2,HGC_L1,HGC_L2,Valid_Day
50897,2017-08-09 23:55:07,234.0,True,False,False,False,False,False,True
50898,2017-08-09 23:50:07,227.0,True,False,False,False,False,False,True
50899,2017-08-09 23:45:07,220.0,True,False,False,False,False,False,True
50900,2017-08-09 23:40:07,218.0,True,False,False,False,False,False,True
50901,2017-08-09 23:35:07,219.0,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...
51082,2017-08-09 08:30:05,177.0,False,False,True,False,False,False,True
51083,2017-08-09 08:25:05,175.0,False,False,True,False,False,False,True
51084,2017-08-09 08:20:05,176.0,False,False,True,False,False,False,True
51085,2017-08-09 08:15:05,177.0,False,False,True,False,False,False,True


In [141]:
set(cgm_DayTime_Aut['Date_Time'].dt.date.unique()) - set(cgm_OvrNight_Aut['Date_Time'].dt.date.unique())

{datetime.date(2017, 8, 9)}

In [142]:
autoOnDateTime

Timestamp('2017-08-09 08:07:13')

In [147]:
print("Len Overnight Manual: " + str(len(cgm_OvrNight_Man)))
print("Len Daytime Manual: " + str(len(cgm_DayTime_Man)))
print("Len Overnight Auto: " + str(len(cgm_OvrNight_Aut)))
print("Len Daytime Auto: " + str(len(cgm_DayTime_Aut)))
print("Total: "+str(len(cgm_OvrNight_Man)+len(cgm_DayTime_Man)+len(cgm_OvrNight_Aut)+len(cgm_DayTime_Aut)))
# checking if all the rows are extracted
if ( (len(cgm_OvrNight_Man)+len(cgm_DayTime_Man) != len(cgm_WholeDay_Man)) or
     (len(cgm_OvrNight_Aut)+len(cgm_DayTime_Aut) != len(cgm_WholeDay_Aut)) ):
    print("ERROR: The length of extracted subsegments of data DP NOT mactch.")

Len Overnight Manual: 936
Len Daytime Manual: 2615
Len Overnight Auto: 10865
Len Daytime Auto: 32730
Total: 47146


STEP 7: use groupby and count on overnight, daytime and whole day segments
(Work on Each metric one-by-one through a loop)

In [164]:
cgm_OvrNight_Man.groupby(pd.Grouper(key='Date_Time', freq='1D'))[HGC_Ranges[3]].apply(lambda x: ((sum(x)/288)*100) )

Date_Time
2017-07-26    19.791667
2017-07-27     0.000000
2017-07-28    21.875000
2017-07-29     6.597222
2017-07-30    11.111111
2017-07-31     0.000000
2017-08-01     9.027778
2017-08-02    22.222222
2017-08-03     0.000000
2017-08-04    22.222222
2017-08-05     0.000000
2017-08-06    18.750000
2017-08-07    21.180556
2017-08-08    18.055556
2017-08-09    11.111111
Freq: D, Name: HGC_R2, dtype: float64

In [148]:
# list variables to store manual and auto values
# the value at index 0 is OVERNIGHT HGC (CGM > 170)
#              index 1 is OVERNIGHT HGC_C (CGM > 250)
#              index 2 is OVERNIGHT HGC_R1 (CGM between 70 and 180 inclusive)
#              index 3 is OVERNIGHT HGC_R2 (CGM between 70 and 150 inclusive)
#              index 4 is OVERNIGHT HGC_L1 (CGM < 70)
#              index 5 is OVERNIGHT HGC_L2 (CGM < 54)
# indices 6 to 11 are same ranges but for DAYTIME
# indices 12 to 17 are same ranges but for WHOLE DAY
man_hgc_vals = []
aut_hgc_vals = []

# inserting OVERNIGHT values
for hgc_type in HGC_Ranges:
    # MANUAL
    ovrNight_Man_Perc = cgm_OvrNight_Man.groupby(pd.Grouper(key='Date_Time', freq='1D'))[hgc_type].apply(lambda x: ((sum(x)/288)*100) if len(x)>0 else np.nan )
    # average over all valid days
    ovrNight_Man_Avg = np.mean(ovrNight_Man_Perc.dropna())
    #print("MANUAL OVERNIGHT "+hgc_type+" = "+str(ovrNight_Man_Avg))
    # add to the list
    man_hgc_vals.append(ovrNight_Man_Avg)
    
    # AUTO
    ovrNight_Aut_Perc = cgm_OvrNight_Aut.groupby(pd.Grouper(key='Date_Time', freq='1D'))[hgc_type].apply(lambda x: ((sum(x)/288)*100) if len(x)>0 else np.nan )
    # average over all valid days
    ovrNight_Aut_Avg = np.mean(ovrNight_Aut_Perc.dropna())
    #print("AUTO OVERNIGHT "+hgc_type+" = "+str(ovrNight_Aut_Avg))
    # add to the list
    aut_hgc_vals.append(ovrNight_Aut_Avg)
    
# inserting DAYTIME values
for hgc_type in HGC_Ranges:
    # MANUAL
    dayTime_Man_Perc = cgm_DayTime_Man.groupby(pd.Grouper(key='Date_Time', freq='1D'))[hgc_type].apply(lambda x: ((sum(x)/288)*100) if len(x)>0 else np.nan )
    # average over all valid days
    dayTime_Man_Avg = np.mean(dayTime_Man_Perc.dropna())
    #print("MANUAL DAYTIME "+hgc_type+" = "+str(dayTime_Man_Avg))
    # add to the list
    man_hgc_vals.append(dayTime_Man_Avg)
    
    # AUTO
    dayTime_Aut_Perc = cgm_DayTime_Aut.groupby(pd.Grouper(key='Date_Time', freq='1D'))[hgc_type].apply(lambda x: ((sum(x)/288)*100) if len(x)>0 else np.nan )
    # average over all valid days
    dayTime_Aut_Avg = np.mean(dayTime_Aut_Perc.dropna())
    #print("AUTO DAYTIME "+hgc_type+" = "+str(dayTime_Aut_Avg))
    # add to the list
    aut_hgc_vals.append(dayTime_Aut_Avg)
    
# inserting WHOLE DAY values
for hgc_type in HGC_Ranges:
    # MANUAL
    wholeDay_Man_Perc = cgm_WholeDay_Man.groupby(pd.Grouper(key='Date_Time', freq='1D'))[hgc_type].apply(lambda x: ((sum(x)/288)*100) if len(x)>0 else np.nan )
    # average over all valid days
    wholeDay_Man_Avg = np.mean(wholeDay_Man_Perc.dropna())
    #print("MANUAL WHOLE DAY "+hgc_type+" = "+str(wholeDay_Man_Avg))
    # add to the list
    man_hgc_vals.append(wholeDay_Man_Avg)
    
    # AUTO
    wholeDay_Aut_Perc = cgm_WholeDay_Aut.groupby(pd.Grouper(key='Date_Time', freq='1D'))[hgc_type].apply(lambda x: ((sum(x)/288)*100) if len(x)>0 else np.nan )
    # average over all valid days
    wholeDay_Aut_Avg = np.mean(wholeDay_Aut_Perc.dropna())
    #print("AUTO WHOLE DAY "+hgc_type+" = "+str(wholeDay_Aut_Avg))
    # add to the list
    aut_hgc_vals.append(wholeDay_Aut_Avg)

In [149]:
#print("MANUAL MODE VALUES: ")
#print(man_hgc_vals)
#print("AUTO MODE VALUES: ")
#print(aut_hgc_vals)

STEP 8: convert results into 2x18 matrix and create Results.csv file for it

In [150]:
# create a 2x18 matrix from the two lists
results_matrix = np.row_stack((man_hgc_vals, aut_hgc_vals))

In [151]:
# create a CSV file of the results
np.savetxt('Results.csv', results_matrix, delimiter=',')

In [152]:
results_matrix

array([[ 4.6474359 ,  0.77457265, 18.48290598, 13.9957265 ,  0.77457265,
         0.        , 24.70619658,  8.76068376, 39.1025641 , 30.42200855,
         3.47222222,  1.5758547 , 29.35363248,  9.53525641, 57.58547009,
        44.41773504,  4.24679487,  1.5758547 ],
       [ 2.77777778,  0.42540471, 20.63097866, 17.93598234,  0.62086093,
         0.15636497, 20.78993056,  5.05071272, 47.5717288 , 35.54915936,
         3.57501827,  1.08506944, 23.54943348,  5.47331871, 68.06697734,
        53.36714181,  4.19179459,  1.2404057 ]])